In [1]:
import numpy as np
import pickle

In [2]:
##データの読み込み
DATA_PATH='./'

f=open(DATA_PATH+"/train_data201604_without_normalize_2.txt","rb")
pre_pre_data=pickle.load(f)
#別に取得したデータを結合させる(201604のデータ)
f=open(DATA_PATH+"/train_data201606_without_normalize_2.txt","rb")
extend_data=pickle.load(f)


f=open(DATA_PATH+"/train_label201604_2.txt","rb")
label=pickle.load(f)
##別に取得したデータを結合させる(201604のデータ)
f=open(DATA_PATH+"/train_label201606_2.txt","rb")
extend_label=pickle.load(f)

In [3]:
print("origine length: %d" % len(pre_pre_data))
print("extend length: %d" % len(extend_data))

##追加データを配列に加える処理
##特徴量
for i in range(len(extend_data)):
    pre_pre_data.append(extend_data[i])
##正解ラベル
for i in range(len(extend_label)):
    label.append(extend_label[i])

#####データの長さの確認
print("append data: %d" % len(pre_pre_data))

origine length: 133215
extend length: 133215
append data: 266430


In [4]:
print(len(pre_pre_data))
print(len(label))
print(sum(label)/len(label))

266430
266430
0.307923281913


In [5]:
##全データのうちの最大値をとるindexを確認
p_data=np.array(pre_pre_data)
max_index=np.argwhere(p_data == p_data.max())
print(max_index)

[[ 11136     10      7]
 [144351     10      7]]


In [6]:
##すべて0のデータがないか確認
zero_index=[]
for i in range(len(p_data)):
    if p_data[i].max == 0:
        zero_index.append(1)
    else:
        zero_index.append(0)
sum(zero_index)

0

In [7]:
##だいたいどれくらい以上の値で全データのうちに占めるのかの計算
len(np.argwhere(p_data>140))/len(p_data)

0.09316518410088954

In [8]:
###極端な値が含まれているデータを除いて新しいデータセットを作ってみる
##ここでは、外れ値として上位約9%の値を除いたデータセットにする
pp_data=[]
pp_label=[]
for i in range(len(pre_pre_data)):
    if (np.max(pre_pre_data[i]) <=140 ):
        pp_data.append(pre_pre_data[i])
        pp_label.append(label[i])
    
    else:
        continue

In [9]:
##特徴量と正解ラベルのデータを同じindexで並べ替える
random_index=[]
for i in range(len(pp_data)):
    random_index.append(i)
    
##random_indexをシャッフルする
random_index=np.random.permutation(random_index)

##random_indexを使って、新しいデータ配列を作成する
pp_data2=[]
pp_label2=[]
for i in random_index:
    pp_data2.append(pp_data[i])
    pp_label2.append(pp_label[i])

#### 各画像ごとの各列の正解ラベルに対しての合計をnumpyで格納して、各列の正解ラベルに対しての相関係数を求める

In [10]:
soukan=[]
num_sum=[]

soukan_pre_data=np.array(pp_data2)
soukan_label=np.array(pp_label2)

sum_list = [[] for i in range(11)]
col_list = [[] for i in range(11)]
##各ユーザーごとに各列の合計を取ってくる
for i in range(len(pp_data2)):
    ##i番目のユーザーの各列の合計
    num_sum=soukan_pre_data[i].sum(axis=0)
    for j in range(len(num_sum)):
        sum_list[j].append(num_sum[j])
##各特徴量を正解ラベルに対して相関を取る        
for i in range(11):
    col_list[i].append(np.corrcoef(sum_list[i] , soukan_label)[0][1])

##相関の高いindex順に並べ替える
num_list=[]
for i in range(len(col_list)):
    num = np.argsort(np.array(col_list).flatten())[::-1][i]
    num_list.append(num)

In [11]:
print(col_list)
print(num_list)
##指定した順に列を並べ替える
pp_data3=[]

for i in range(len(pp_data2)):
    data=pp_data2[i][:, num_list]
    pp_data3.append(data)

[[0.04196269420740166], [0.034791228451347252], [0.17801460776674793], [0.0057495750408455143], [0.019707631682382036], [0.049351184300400149], [0.11703170244132817], [0.012661345003537869], [0.065938722149423493], [0.1488590251135852], [0.017949394562392466]]
[2, 9, 6, 8, 5, 0, 1, 4, 10, 7, 3]


In [12]:
'''0以外のところを上限256として反転させる'''
pp_reverse=[]
for i in range(len(pp_data3)):
    add_data=np.where(pp_data3[i]>0 ,256-pp_data3[i],pp_data3[i])
    pp_reverse.append(add_data)

In [13]:
import sys
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|              extend_data|   1080112|
|             extend_label|   1080112|
|                    label|   2190072|
|                   p_data| 703375328|
|                  pp_data|   2115952|
|                 pp_data2|   2115952|
|                 pp_data3|   2115952|
|                 pp_label|   2115952|
|                pp_label2|   2115952|
|               pp_reverse|   2115952|
|             pre_pre_data|   2190072|
|             random_index|   2022800|
|             soukan_label|   1011448|
|          soukan_pre_data| 667492448|
|               zero_index|   2380488|


In [14]:
import gc
del label,p_data,pp_data,pp_data2,pp_data3,pp_label,pre_pre_data,soukan_label,soukan_pre_data,zero_index
gc.collect()

529

#### テストデータだけ切り離し、標準化して保存

In [15]:
test_buy=[]
test_nobuy=[]
test_index=[]
x_test_add_noise=[]
y_test_add_noise=[]
length=int(len(pp_reverse)*0.1)
for i in range(length):
    index=int(np.random.choice(random_index,1))
    test_index.append(index)
    
for i in test_index:
    x_test_add_noise.append(pp_reverse[i])
    y_test_add_noise.append(pp_label2[i])
    
    
train_index=list(set(random_index)-set(test_index))
###正例、不例のサンプリングをする
##正例、不例のindexを取得しておく

buy=[]
no_buy=[]
for i in train_index:
    if pp_label2[i]==1:
        buy.append(i)
    else:
        no_buy.append(i)
print('positive data:{}'.format(len(buy)))
print('negative data:{}'.format(len(no_buy)))

positive data:67971
negative data:160833


In [ ]:
###train
train_buy=[]
train_nobuy=[]
length=int(len(pp_reverse)*0.9*0.9)
for i in range(length):
    index=int(np.random.choice(buy,1))
    train_buy.append(index)
for i in range(length):
    index=int(np.random.choice(no_buy,1))
    train_nobuy.append(index)
    
buy2=list(set(buy)-set(train_buy))
no_buy2=list(set(no_buy)-set(train_nobuy))

##train
x_train_add_noise=[]
y_train_add_noise=[]

valid_x_test_add_noise=[]
valid_y_test_add_noise=[]

for i in train_buy:
    x_train_add_noise.append(pp_reverse[i])
    y_train_add_noise.append(pp_label2[i])
for i in train_nobuy:
    x_train_add_noise.append(pp_reverse[i])
    y_train_add_noise.append(pp_label2[i])
    
for i in buy2:
    valid_x_test_add_noise.append(pp_reverse[i])
    valid_y_test_add_noise.append(pp_label2[i])
for i in no_buy2:
    valid_x_test_add_noise.append(pp_reverse[i])
    valid_y_test_add_noise.append(pp_label2[i])

def scale(X):
    """データ行列Xを属性ごとに標準化したデータを返す"""
    # 属性の数 (列の数)
    delta=1e-3
    col = X[0].shape[1]

   
   # 属性ごとにデータを標準化or正則化
    for j in range(len(X)):
        for i in range(col):
            # 属性ごとに平均値と標準偏差を計算
            mu = np.mean(X[j][:,[i]])
            sigma = np.std(X[j][:,[i]])
            X[j][:,[i]]= (X[j][:,[i]]- mu) / (sigma+delta)
            
    return X
  
##各ユーザーデータを列ごとに標準化(列の値が全て0の場合を考えて、分母に極小の値を加えて標準化)
pp_data_np=np.asarray(x_test_add_noise)
pre_data2=scale(pp_data_np)

In [ ]:
'''
標準化の際にnanが生じている場合に備えてnan削除の処理
'''
np.set_printoptions(threshold=np.inf)
len(np.unique(np.where(np.isnan(pre_data2))[0]))
nan_list=np.unique(np.where(np.isnan(pre_data2))[0])

##nanの部分を削除
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]
pre_data3=dellist(pre_data2,nan_list)
label=dellist(y_test_add_noise,nan_list)

'''特徴量:pre_data3   ラベル:label'''
##データの長さがずれないか確認
print(len(label))
print(len(pre_data3))

In [ ]:
##test dataの保存
import pickle
f=open("./x_test_corr_high_test_without_add_some_noise_samesample.txt","wb")
pickle.dump(pre_data3, f)
f=open("./y_test_corr_high_test_without_add_some_noise_samesample.txt","wb")
pickle.dump(label, f)

print(sum(label)/len(label))

In [ ]:
##validation 標準化
pp_data_np=np.asarray(valid_x_test_add_noise)
pre_data2=scale(pp_data_np)

np.set_printoptions(threshold=np.inf)
len(np.unique(np.where(np.isnan(pre_data2))[0]))
nan_list=np.unique(np.where(np.isnan(pre_data2))[0])

##nanの部分を削除
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]
pre_data3=dellist(pre_data2,nan_list)
label=dellist(valid_y_test_add_noise,nan_list)

'''特徴量:pre_data3   ラベル:label'''
##データの長さがずれないか確認
print(len(label))
print(len(pre_data3))
print(sum(label)/len(label))

##valid
f=open("./validation_x_test_corr_high_test_without_add_some_noise_samesample.txt","wb")
pickle.dump(pre_data3, f)
f=open("./validation_y_test_corr_high_test_without_add_some_noise_samesample.txt","wb")
pickle.dump(label, f)

In [ ]:
###正例、不例のサンプリングをする
##正例、不例のindexを取得しておく

buy=[]
no_buy=[]
for i in range(len(y_train_add_noise)):
    if y_train_add_noise[i]==1:
        buy.append(i)
    else:
        no_buy.append(i)
print('positive data:{}'.format(len(buy)))
print('negative data:{}'.format(len(no_buy)))

### データ水増し

In [ ]:
'''データの水増しを行う
不例を20万枚増やして正例データを同じ数になるようにしてみる
正例、不例ともにごま塩加工で増やす
'''
##塩の割合を指定
s_vs_p = 0.7
##全体的にごま塩を増やしたいなら、ここを変える
amount = 0.15
from skimage import transform
import random


'''正例、不例ともに画素15%にごま塩加工'''
salt_papper_posi_x=[]
salt_papper_posi_y=[]
salt_papper_nega_x=[]
salt_papper_nega_y=[]

salt_papper_posi_x2=[]
salt_papper_posi_y2=[]
salt_papper_nega_x2=[]
salt_papper_nega_y2=[]

no_buy_loop = 200000-len(no_buy)

'''負例データ'''
for j in range(no_buy_loop):
    ##データindexのランダム選択
    nega=int(np.random.choice(no_buy,1))
    src = x_train_add_noise[nega]
    sp_img = src.copy()
    # 塩モード
    num_salt = np.ceil(amount * src.size * s_vs_p)
    coords = [np.random.randint(0, i , int(num_salt)) for i in sp_img.shape]
    ##代入する値をランダムに選択(200から255)
    input_number = int(np.random.randint(116, 256 ,1))
    sp_img[coords] = input_number

    # 胡椒モード
    num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i , int(num_pepper)) for i in sp_img.shape]
    sp_img[coords] = 0
    
    salt_papper_nega_x.append(sp_img)
    salt_papper_nega_y.append(y_train_add_noise[nega])
    


buy_loop = 200000-len(buy)

'''正例データ'''
for j in range(buy_loop):
    ##データindexのランダム選択
    posi=int(np.random.choice(buy,1))
    src = x_train_add_noise[posi]
    sp_img = src.copy()
    # 塩モード
    num_salt = np.ceil(amount * src.size * s_vs_p)
    coords = [np.random.randint(0, i , int(num_salt)) for i in sp_img.shape]
    ##代入する値をランダムに選択(200から255)
    input_number = int(np.random.randint(116, 256 ,1))
    sp_img[coords] = input_number

    # 胡椒モード
    num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i , int(num_pepper)) for i in sp_img.shape]
    sp_img[coords] = 0
    
    salt_papper_posi_x.append(sp_img)
    salt_papper_posi_y.append(y_train_add_noise[posi])
    


In [ ]:
###データappend
for i in range(len(salt_papper_posi_x)):
    x_train_add_noise.append(salt_papper_posi_x[i])
    y_train_add_noise.append(salt_papper_posi_y[i])
    
for i in range(len(salt_papper_nega_x)):
    x_train_add_noise.append(salt_papper_nega_x[i])
    y_train_add_noise.append(salt_papper_nega_y[i])


### 標準化

In [ ]:
##各ユーザーデータを列ごとに標準化(列の値が全て0の場合を考えて、分母に極小の値を加えて標準化)
pp_data_np=np.asarray(x_train_add_noise)
pre_data2=scale(pp_data_np)

In [ ]:
'''
標準化の際にnanが生じている場合に備えてnan削除の処理
'''

np.set_printoptions(threshold=np.inf)
len(np.unique(np.where(np.isnan(pre_data2))[0]))
nan_list=np.unique(np.where(np.isnan(pre_data2))[0])

##nanの部分を削除
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]
pre_data3=dellist(pre_data2,nan_list)
label=dellist(y_train_add_noise,nan_list)

'''特徴量:pre_data3   ラベル:label'''
##データの長さがずれないか確認
print(len(label))
print(len(pre_data3))

In [ ]:
##train保存
##train
f = open('./validation_x_train_corr_high_test_without_add_some_noise_samesample.txt', 'wb')
pickle.dump(pre_data3, f)
f = open('./validation_y_train_corr_high_test_without_add_some_noise_samesample.txt', 'wb')
pickle.dump(label, f)


In [ ]:
print(sum(label)/len(label))